<a href="https://colab.research.google.com/github/lidar532/ppkgeotag/blob/master/mp_genjpg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import os

bash_script_fn = 'generate-jpgs.txt'
jpg_quality = 50
idir = '/usr/bin/'
odir = '/tmp/'
#idir = 'j:/data/2020-0208-NC-outer-banks/dng'
#odir = '/mnt/j/data/2020-0208-NC-outer-banks/field-jpg'

threads = os.cpu_count()				# Determine the number of CPU cores 
allfiles = os.listdir(idir)			# gather list of input photo files

tc = len(allfiles)
step = int(tc/threads)
blocksz = step*threads

hs = f'\
# Generate jpgs with multiple gm.  W. Wright 2020-0217\n\
#########################################################################\n\
#         bash Script: {bash_script_fn}\n\
#           Input dir: {idir}\n\
#          Output dir: {odir}\n\
# CPU Cores (threads):{threads:2d}  Jpeg Quality:{jpg_quality:2d}   Blocksize:{blocksz:3d}\n\
#    Total File Count:{tc:4d}        Step: {step:4d} \n\
#########################################################################\n\
# \n'

print(hs)						# Display settings on the console

# Generate a Linux Bash script file containing the commands
# to execute graphicsmagick commands to convert an input file list
# into a new directory of converted images.
bash_script = open( bash_script_fn, 'w')
bash_script.write(hs)
bash_script.write(f'cd {idir}\n')

starts = np.arange(0, tc, step)
stops  = (starts -1)[1:]
blocks = []

print(starts,"\n", stops, len(starts), " ", len(stops))
for i in range( len(stops)):
		print(f'{i}   {starts[i]}  {stops[i]}   {stops[i] - starts[i]}')    # 
		bk = allfiles[starts[i]:stops[i]]                                   # Extract slice 
		blocks.append(bk)
### print(f'Last   {starts[-1]}  {tc}  {tc - starts[-1]}')
blocks.append(allfiles[starts[-1]:tc])

# Each block contains a list of files to pass to a thread for processing
core_number = 0
for i in blocks:
	fn = str(f'f-{core_number}.txt')
	bash_script.write(str(f'cat {fn} | xargs gm mogrify -create-directories -output-directory {odir} -format jpeg -quality {jpg_quality} - &\n'))
	with open(fn, 'w') as f:
		for n in i:
			f.write(str(f'{n} '))
	print(f'File: {fn} generated.')
	print(i[0:4])
	core_number = core_number + 1

bash_script.write('echo \'Conversion scripts started\'\n')
bash_script.close()
  
  